## Transformer 번역기 성능 개선 실험 회고

### Back-translation 및 Mecab 도입

- Transformer 베이스라인 성능 개선을 위해 NLLB 모델을 활용한 Back-translation(역번역) 데이터 증강을 시도했다. 학습 데이터의 10%만 증강했음에도 Validation Loss가 4.597까지 감소하는 것을 보며 데이터 증강의 가능성을 확인했다.

- Back-translation 적용 이후에 한국어의 형태론적 특성을 더 잘 반영하고자 Mecab 형태소 분석기를 도입했다. 하지만 끝까지 훈련을 완료하고 정량적 성능 지표를 비교하지는 못해 아쉬움이 남았다.

- 역번역 효과를 극대화하고자 데이터셋 전체(100%)를 증강하는 실험을 계획했으나, 예상보다 훈련 시간이 매우 길어져 리소스 문제로 실험을 끝까지 진행하지 못했다. 대규모 데이터 증강의 효과를 직접 확인하지 못한 점이 이번 프로젝트의 가장 아쉬운 부분이다.


### 어휘 대체(Lexical Substitution) 및 학습 안정화 (v1.0 ~ v1.2)

- 역번역 외 다른 증강 기법으로 Word2Vec을 이용한 어휘 대체(Lexical Substitution)를 시도했다(v1.0). 이후 자체 학습 모델 대신 사전 학습된 ko.vec 임베딩을 사용하고, 학습률 스케줄러를 적용(v1.1)하며 Validation Loss가 4.105까지 감소하는 유의미한 성능 개선을 이끌어냈다.

- 퍼실님께서 조언해주신 부분을 바탕으로 체크포인팅(Checkpointing)과 조기 종료(Early Stopping)를 도입했다(v1.2). 그 결과, 불필요한 학습을 막고 patience=3 설정에 따라 12번째 에폭에서 훈련을 자동으로 중단시켜, 시간과 컴퓨팅 자원을 효율적으로 관리할 수 있게 되었다. 이는 안정적인 실험 환경 구축의 중요성을 깨닫게 된 계기가 되었다.

### 최신 기법 집약 및 종합 평가 (v2.0)

- 최종 버전(v2.0)에서는 이전 실험에서 효과가 검증된 NLLB 역번역, Mecab 토크나이저를 모두 적용했다. 여기에 더해 AdamW, Label Smoothing, 코사인 어닐링 스케줄러와 같은 최신 훈련 기법들을 집약했다.

- 그 결과 Validation Loss가 3.595까지 감소하며 가장 좋은 성능을 달성했다. 이는 단일 기법의 도입보다 검증된 여러 기법들의 시너지가 최종 성능에 결정적인 영향을 미친다는 것을 보여준다.

- 특히 이전 실험들에서 Loss 외에는 성능을 판단할 지표가 부재했던 문제를 해결하기 위해, BLEU, ROUGE, BERTScore 등 다양한 지표를 포함한 종합 평가 파이프라인을 구축했다. 이를 통해 모델의 번역 품질을 다각적으로 분석할 수 있는 체계를 마련할 수 있었다.

### 실험 소감 및 향후 계획

- 이번 실험은 베이스라인 모델에서 출발하여 다양한 기법을 점진적으로 추가하면서 각 요소의 효과를 체계적으로 검증하는 것을 목표로 했다.

- 하지만 생각보다 시간이 너무 오래 걸려서(특히 역번역) 팀원들과 함께 실험 결과를 정리할 시간이 많이 부족했다.

- 다음에는 제한된 시간을 더 잘 고려하여 실험의 범위(scope)를 명확히 설정하고, 가설 검증부터 시사점 도출까지 하나의 사이클을 완전히 마무리할 수 있도록 분발해야겠다.